In [1]:
import os
import unittest
import time
import copy
from pymatgen.core.structure import Molecule
from pymatgen.analysis.graphs import MoleculeGraph
from pymatgen.analysis.local_env import OpenBabelNN
from pymatgen.util.testing import PymatgenTest
from pymatgen.analysis.reaction_network.reaction_network import ReactionNetwork
from pymatgen.analysis.fragmenter import Fragmenter, metal_edge_extender
from pymatgen.entries.mol_entry import MoleculeEntry
from monty.serialization import loadfn,dumpfn
import openbabel as ob
import networkx as nx

In [2]:
from crystal_toolkit.helpers.pythreejs_renderer import view_old

In [3]:
# prod_entries = []
# entries = loadfn("/Users/samuelblau/Desktop/smd_production_entries.json")
# for entry in entries:
#     if "optimized_molecule" in entry["output"]:
#         mol = entry["output"]["optimized_molecule"]
#     else:
#         mol = entry["output"]["initial_molecule"]
#     H = float(entry["output"]["enthalpy"])
#     S = float(entry["output"]["entropy"])
#     E = float(entry["output"]["final_energy"])
#     mol_entry = MoleculeEntry(molecule=mol,energy=E,enthalpy=H,entropy=S,entry_id=entry["task_id"])
#     prod_entries.append(mol_entry)

In [5]:
# RN = ReactionNetwork(prod_entries)
RN = loadfn("prod_RN.json")
# dumpfn(RN,"prod_RN.json")

8516 input entries
7161 connected entries
5927 unique entries
Missing H3O+, will not add concerted water splitting rxn1
Adding concerted water splitting rxn2: 2H2O + 2e- -> H2 + 2OH-
Water rxn2 free energy = 0.9484209174741407


In [7]:
EC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/EC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
EC_mg = metal_edge_extender(EC_mg)

FEC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/FEC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
FEC_mg = metal_edge_extender(FEC_mg)

H2O_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/H2O.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
H2O_mg = metal_edge_extender(H2O_mg)

LEMC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/LEMC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEMC_mg = metal_edge_extender(LEMC_mg)

LEDC_mg =  MoleculeGraph.with_local_env_strategy(
    Molecule.from_file("../../../../test_files/reaction_network_files/LEDC.xyz"),
    OpenBabelNN(),
    reorder=False,
    extend_structure=False)
LEDC_mg = metal_edge_extender(LEDC_mg)

EC_ind = None
FEC_ind = None
LEDC_ind = None
LEMC_ind = None
H2O_ind = None
for entry in RN.entries["C3 H4 O3"][10][0]:
    if EC_mg.isomorphic_to(entry.mol_graph):
        EC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 F1 H3 O3"][10][0]:
    if FEC_mg.isomorphic_to(entry.mol_graph):
        FEC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C4 H4 Li2 O6"][17][0]:
    if LEDC_mg.isomorphic_to(entry.mol_graph):
        LEDC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["C3 H5 Li1 O4"][13][0]:
    if LEMC_mg.isomorphic_to(entry.mol_graph):
        LEMC_ind = entry.parameters["ind"]
        break
for entry in RN.entries["H2 O1"][2][0]:
    if H2O_mg.isomorphic_to(entry.mol_graph):
        H2O_ind = entry.parameters["ind"]
        break
Li1_ind = RN.entries["Li1"][0][1][0].parameters["ind"]

print("EC_ind",EC_ind)
print("FEC_ind",FEC_ind)
print("LEDC_ind",LEDC_ind)
print("LEMC_ind",LEMC_ind)
print("Li1_ind",Li1_ind)
print("H2O_ind",H2O_ind)

EC_ind 2815
FEC_ind 1631
LEDC_ind 4147
LEMC_ind 2870
Li1_ind 5901
H2O_ind 5897


In [8]:
ind=1631
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.741831367760001, left=-4.741831367760001, nea…

0


-12018.648253254347

In [9]:
for neighbor in list(RN.graph.neighbors(1631)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

1631,1630
rxn_type : One electron reduction
bipartite : 1
energy : -0.041393790765027916
free_energy : 0.9063516963273286

1631,1632
rxn_type : One electron oxidation
bipartite : 1
energy : 0.3359276730519696
free_energy : 6.800411779775095

1631+PR_5899,1561
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.0013195908420122748
free_energy : 0.24703608665768684

1631+PR_5900,1566
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.017374984524039405
free_energy : -0.1735480704583665

1631+PR_5901,1568
rxn_type : Molecular formation from one new bond A+B -> C
bipartite : 1
energy : -0.03201866546403043
free_energy : -0.5487826012532651

1631,1514+5882
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.2411844358719577
free_energy : 5.941007637577059

1631,1512+5883
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.17085020192200773
free_energy : 3.98

In [24]:
ind=1568
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-6.83031684216, left=-6.83031684216, near=-2000.…

1


-12220.688743195993

In [11]:
ind=1571
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.777487102639999, left=-4.777487102639999, nea…

0


-12223.018963532839

In [12]:
def red_pot(pos_ind,neut_ind):
    return RN.entries_list[pos_ind].free_energy-RN.entries_list[neut_ind].free_energy-1.4

In [13]:
print(red_pot(1568,1571))

0.9302203368457413


In [14]:
for neighbor in list(RN.graph.neighbors(1568)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

1568,1566
rxn_type : One electron reduction
bipartite : 1
energy : -0.07327232616000856
free_energy : 0.13291777435406393

1568,1617+5904
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.37413205707696306
free_energy : 9.559487822029041

1568,1618+5903
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.2966416465224029
free_energy : 7.488893666297372

1568,1397+5884
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.3221836828650453
free_energy : 8.391487034316015

1568,1404+5883
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.17471867073703606
free_energy : 4.080645634237953

1568,1631+5901
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.03201866546402243
free_energy : 0.5487826012540609

1568,1632+5900
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.28003033141

In [15]:
for neighbor in list(RN.graph.neighbors(1566)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

1566,1561
rxn_type : One electron reduction
bipartite : 1
energy : -0.05335654801797318
free_energy : 0.6817887408600654

1566,1568
rxn_type : One electron oxidation
bipartite : 1
energy : 0.07327232616000856
free_energy : -0.13291777435406393

1566,1551
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.07873140279701829
free_energy : 1.9005103800955112

1566,1553
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : -0.022451995094968424
free_energy : -0.7570817213854752

1566,1571
rxn_type : Intramolecular single bond formation
bipartite : 1
energy : -0.01088540933500326
free_energy : -0.3131381111998053

1566,1630+5901
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.06389720085900308
free_energy : 1.3222165232273255

1566,1631+5900
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.017374984524053616
free_energy : 0.17354807045921916

1566,1632+5899
rxn_type : Molecular d

In [25]:
ind=1566
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-6.179342544599999, left=-6.179342544599999, nea…

0


-12222.705825421639

In [41]:
view_old(RN.entries["C3 F1 H3 Li1 O3"][12][0][0].molecule,bonding_strategy="OpenBabelNN")

Renderer(background='white', camera=OrthographicCamera(bottom=-4.777487102639999, left=-4.777487102639999, nea…

In [42]:
RN.entries["C3 F1 H3 Li1 O3"][12][0][0]

MoleculeEntry 135896 - C3 F1 H3 Li1 O3 - E12 - C0
Energy = -449.2236 Hartree
Correction = 0.0000 Hartree
Enthalpy = 47.3820 kcal/mol
Entropy = 83.0950 cal/mol.K
Free Energy = -12223.0190 eV
Parameters:
ind = 1571

In [71]:
for neighbor in list(RN.graph.neighbors(1571)):
    print(neighbor)
    for key in RN.graph.node[neighbor]:
        print(key,":",RN.graph.node[neighbor][key])
    print()

1571,1565
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : -0.034118205551976644
free_energy : -1.0934727998792368

1571,1566
rxn_type : Intramolecular single bond breakage
bipartite : 1
energy : 0.01088540933500326
free_energy : 0.3131381111998053

1571,1409+5883
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.105657558353073
free_energy : 2.2666601754681324

1571,1407+5884
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.3515147306120525
free_energy : 9.196273763840509

1571,2413+5822
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.7318361085210086
free_energy : 19.39406840511947

1571,2415+5821
rxn_type : Molecular decomposition breaking one bond A -> B+C
bipartite : 1
energy : 0.141138023075996
free_energy : 3.3723394811095204

1571,1630+5901
rxn_type : Coordination bond breaking AM -> A+M
bipartite : 1
energy : 0.07478261019400634
free_energy : 1

In [45]:
ind=1565
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-7.35967406268, left=-7.35967406268, near=-2000.…

0


-12224.112436332718

In [69]:
ind=1564
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-7.426678041599999, left=-7.426678041599999, nea…

0


-12223.857551716179

In [75]:
RN.entries["C3 F1 H3 Li1 O3"][12][0]

[MoleculeEntry 135896 - C3 F1 H3 Li1 O3 - E12 - C0
 Energy = -449.2236 Hartree
 Correction = 0.0000 Hartree
 Enthalpy = 47.3820 kcal/mol
 Entropy = 83.0950 cal/mol.K
 Free Energy = -12223.0190 eV
 Parameters:
 ind = 1571]

In [74]:
RN.entries_list[1564]

MoleculeEntry 136447 - C3 F1 H3 Li1 O3 - E11 - C0
Energy = -449.2492 Hartree
Correction = 0.0000 Hartree
Enthalpy = 46.3570 kcal/mol
Entropy = 90.6470 cal/mol.K
Free Energy = -12223.8576 eV
Parameters:
ind = 1564

In [76]:
ind=1571
view_old(RN.entries_list[ind].molecule,bonding_strategy="OpenBabelNN")
print(RN.entries_list[ind].charge)
RN.entries_list[ind].free_energy

Renderer(background='white', camera=OrthographicCamera(bottom=-4.777487102639999, left=-4.777487102639999, nea…

0


-12223.018963532839